In [1]:
import popart
import onnx 
import numpy as np
from copy import deepcopy
from onnx.tools import update_model_dims

In [2]:
bert_model = onnx.load("bertsquad-10/bertsquad10.onnx")

In [3]:
onnx.checker.check_model(bert_model, 6)

In [ ]:
bert_model.opset_import

In [ ]:
bert_model.graph.output

In [ ]:
graph_transformer = popart.GraphTransformer("bertsquad-10/bertsquad10.onnx")

In [ ]:
graph_transformer.graph

In [5]:
builder = popart.Builder("bertsquad-10/bertsquad10.onnx")
anchors = {output_name: popart.AnchorReturnType("All") for output_name in builder.getOutputTensorIds() }
dataflow = popart.DataFlow(100, anchors)
device = popart.DeviceManager().createCpuDevice()

In [6]:
session = popart.InferenceSession(bert, dataflow, device)
# session = popart.InferenceSession("bertsquad-10/bertsquad10.onnx", dataflow, device)

In [ ]:
builder.getInputTensorIds()

In [ ]:
builder.getTensorShape('input_ids:0') 

In [ ]:
builder.getTensorDtypeString('input_ids:0')

In [ ]:
'float32'.upper()

In [ ]:
type(bert_model)

In [ ]:
bert_backup = deepcopy(bert_model)

In [ ]:
for input_T in bert_model.graph.input:
    print (input_T.name, end="@ ")
    # get type of input_T tensor
    tensor_type = input_T.type.tensor_type
    # check if it has a shape:
    if (tensor_type.HasField("shape")):
        # iterate through dimensions of the shape:
        for d in tensor_type.shape.dim:
            # the dimension may have a definite (integer) value or a symbolic identifier or neither:
            if (d.HasField("dim_value")):
                print (d.dim_value, end=", ")  # known dimension
            elif (d.HasField("dim_param")):
                print (d.dim_param, end=", ")  # unknown dimension with symbolic name
            else:
                print ("?", end=", ")  # unknown dimension with no name
    else:
        print ("unknown rank", end="")
    print()

In [ ]:
bert_model.graph.input[1].type.tensor_type.shape.dim[0]

In [ ]:
for input_T in bert_model.graph.input:
    print (input_T.name, end="@ ")
    # get type of input_T tensor
    tensor_type = input_T.type.tensor_type
    # check if it has a shape:
    if (tensor_type.HasField("shape")):
        # iterate through dimensions of the shape:
        a, *_ = tensor_type.shape.dim
        if a.HasField("dim_param"):
            tensor_type.shape.dim[0] = a.dim_value

In [ ]:
from onnx.tools import update_model_dims

In [4]:
input_shape = {
    "unique_ids_raw_output___9:0": [3, ],
    "segment_ids:0": [3, 256],
    "input_mask:0": [3, 256],
    "input_ids:0": [3, 256],
    }
output_shape = {
    "unstack:1": [3, 256],
    "unstack:0": [3, 256],
    "unique_ids:0": [3],
    
}
bert = update_model_dims.update_inputs_outputs_dims(bert_model, input_shape, output_shape)

In [ ]:
v.graph.input

In [ ]:
k.graph.output